# Matematický aparát pro RSA

## Eulerova funkce $\phi$
Nechť $c\in \mathbb{N}$, **Eulerova funkce** $\phi$ udává počet čísel $k\in \{1,..,c-1\}$, která jsou nesoudělná s $c$ ( $\gcd(c,k)=1$ ).
$$
\phi(c) = n \Leftrightarrow \\
\exists o\in \widehat{c-1}\ \forall i \in o : |o|=n \wedge \gcd(c,i)=1 
\\ \wedge \\
\forall o\in \widehat{c-1}\ \exists i \in o : |o|>n \rightarrow  \gcd(c,i)\neq 1
$$
Pro výpočet eulerovy funkce lze využít rozkladu na prvočinitele, následujícím způsobem
$$
\phi(c) = \sum_{\forall p \in P} (p-1)
$$
, kde $P$ je množina všech prvnočíslených dělitelů $c$.

---
## Definice (z FIT CVUT)

### Eulerova funkce $\phi(n)$
Eulerova funkce $\phi(n) : \mathbb{N^+} → \mathbb{N^+}$ udává počet kladných celých čı́sel menšı́ch nebo rovných $n$, která jsou nesoudělná s $n$.

### Věta o $\phi(m · n)$
Nechť $m, n \in \mathbb{N}$ a $\gcd(m, n)=1$. Potom $\phi(m · n) = \phi(m) · \phi(n)$

#### Pozorování

Nechť $n = p_1^{\alpha_1} . p_2^{\alpha_2} ... p_k^{\alpha_k}$ je kanonický rozklad složeného čı́sla $n \in \mathbb{N^+}$. Potom $\phi = n(1-\frac{1}{p_1})(1-\frac{1}{p_2})(1-\frac{1}{p_k})$


#### Dusledek
Dusledek 3. vede na vztah $\phi(c) = \sum_{\forall p \in P} (p-1)$
$$
\phi(n) = n * \Pi_{\forall p_i \in P}(1-\frac{1}{p_i}) \\
\phi(n) = n * \Pi(\frac{p_i-1}{p_i}) = n * \Pi(\frac{1}{p_1}) * \Pi(\frac{p_i-1}{1}) = n * \frac{1}{n} *\Pi(\frac{p_i-1}{1})\\
\phi(n) = \Pi_{\forall p_i \in P}(p_i-1)
$$

## Složitost problému faktorizace
Faktorizace čísla na prvočinitele je třídy NP problém

Neexistuje a nepředpokládá se existence algoritmu, který dokáže faktorizaci plnit v polynomiálním čase.

[zdroj](https://stackoverflow.com/a/2642528/10209148)

## Sloitost problému výpočtu eulerovy funkce $\phi(n)$
Výpočtu eulerovy funkce je $O(\sqrt{n})$

**??Zdroj??**

---

## RSA
- Let $p$ and $q$ be large prime numbers.
- Let $n=p*q$
- Choose $e$ be number that satisfy: $1<e<n, \gcd(e,\phi(n))=1$ 
- Let $d=|e^{-1}|_{\Phi(n)}$

Private key is par of $SK=(n,d)$

Public key is par of $PK=(n,e)$

### Process of encription
$c = |m^e|_n$

### Process of decription
$m = |c^d|_n = |(|m^e|_{n})^d|_n = |m^{e^d}|_n = |m^{e*d}|_n = |m^1|_n$

In [13]:
# Vypocet mocnin v modulu
#https://stackoverflow.com/questions/4798654/modular-multiplicative-inverse-function-in-python
def modinv(a, m):
    def egcd(a, b):
        if a == 0:
            return (b, 0, 1)
        else:
            g, y, x = egcd(b % a, a)
            return (g, x - (b // a) * y, y)
    g, x, y = egcd(a, m)
    
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m

In [ ]:
p=297801375689990896481365079840784910951
q=337830217479069994210092116932742172839
n=p*q
e=65_537

phi=(p-1)*(q-1)

d=modinv(e,phi)

In [15]:
print("========")
print(f"Private Key = (n,d) = {(n,d)}")
print(f"Public Key = (n,e) = {(n,e)}")
print("========")

Private Key = (n,d) = (100606303514915852613885507460409931526068760452003533580079596327680865859889, 29781074632488022654521550341516283497770766129993572641013929522858879293473)
Public Key = (n,e) = (100606303514915852613885507460409931526068760452003533580079596327680865859889, 65537)


In [16]:
def crypt(barray: bytes, endianity: str = "little") -> bytes:
    plain=int.from_bytes(barray, endianity)
    crypt = pow(plain,e,n)
    return crypt.to_bytes((crypt.bit_length() + 7) // 8,endianity) \
           or b'\0' #https://stackoverflow.com/a/31761722/10209148


def decrypt(barray: bytes, endianity: str = "little") -> bytes:
    crypt=int.from_bytes(barray, endianity)
    plain = pow(crypt,d,n)
    return plain.to_bytes((plain.bit_length() + 7) // 8,endianity) \
           or b'\0' #https://stackoverflow.com/a/31761722/10209148


In [17]:
message="super_a_cuper_tajny_heslo"

c_text = crypt(message.encode())
print(f"cypher_bytes: {c_text}")
plain = decrypt(c_text)
print(f"plain_bytes: {plain}")
print(f"plain_text: {plain.decode()}")


cypher_bytes: b'k<.\xaf\xf0\xc3\x1cyz\xf8\xae\xd6\x87&]\xab\xc4\xe1j\xb0\xef>2\x97!\xa3\x9bD\xcd\x0e\xd1\x1f'
plain_bytes: b'super_a_cuper_tajny_heslo'
plain_text: super_a_cuper_tajny_heslo


### Bonus
Použitá prvnočísla $p$ a $q$ jsou malá ($\pm 100b$). Toto je bezpečnostní problém, tyto čísla nám nezaručí výpočetní bezpečnost. Ze znalosti veřejného klíče $(e,n)$, lze pomocí faktorizace nálézt čísla $p$ a $q$ a tím dopočítat klíč veřejný.

Následující kód demostruje faktorizaci.

In [18]:
from functools import reduce

def factors(n):    
    return set(reduce(list.__add__, ([i, n//i] for i in range(1, int(n**0.5) + 1) if n % i == 0)))
#factors(n)

---

## TODO
- padding